**I'm taking 2 datasets Human text & Ai text and taking some samples and merging them for a small training session.**

In [68]:
import os

print(os.listdir("/kaggle/input"))

['daigt-hc3-dataset', 'starter-wikitext-103-a0194af6-3']


In [69]:
from datasets import load_dataset

human_ds = load_dataset(
    "wikitext",
    "wikitext-103-raw-v1",
    split="train"
)

print(len(human_ds))
print(human_ds[0])

1801350
{'text': ''}


In [70]:
text_human = human_ds[10]["text"]

print(text_human[:500])
print("Word count:", len(text_human.split()))

 The game 's battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters ' turns . Each character has a field and distance of movement limit
Word count: 296


In [71]:
words = text_human.split()
labels = [0] * len(words)

print("Words:", len(words))
print("Labels:", len(labels))
print("First 10 labels:", labels[:10])

Words: 296
Labels: 296
First 10 labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [72]:
human_samples = []

for item in human_ds:
    text = item["text"]
    wc = len(text.split())

    if wc < 50:
        continue

    human_samples.append({
        "text": text,
        "labels": [0] * wc
    })

    if len(human_samples) == 20:
        break

print("Human samples created:", len(human_samples))

Human samples created: 20


In [73]:
import os
import json

ai_path = "/kaggle/input/daigt-hc3-dataset"

print(os.listdir(ai_path))

['__results__.html', '__resultx__.html', '__notebook__.ipynb', '__results___files', '__output__.json', 'train.csv', 'custom.css']


In [74]:
import pandas as pd
import os

ai_path = "/kaggle/input/daigt-hc3-dataset"

df_ai = pd.read_csv(os.path.join(ai_path, "train.csv"))
print(df_ai.head())
print(df_ai.columns)

                                                text  label
0  Basically there are many categories of " Best ...      0
1  salt is good for not dying in car crashes and ...      0
2  The way it works is that old TV stations got a...      0
3  You ca n't just go around assassinating the le...      0
4  Wanting to kill the shit out of Germans drives...      0
Index(['text', 'label'], dtype='object')


In [75]:
import pandas as pd
import os

ai_path = "/kaggle/input/daigt-hc3-dataset"

df_ai = pd.read_csv(os.path.join(ai_path, "train.csv"))
print(df_ai.head())
print(df_ai.columns)


                                                text  label
0  Basically there are many categories of " Best ...      0
1  salt is good for not dying in car crashes and ...      0
2  The way it works is that old TV stations got a...      0
3  You ca n't just go around assassinating the le...      0
4  Wanting to kill the shit out of Germans drives...      0
Index(['text', 'label'], dtype='object')


In [76]:
print(df_ai["text"].iloc[0][:300])


Basically there are many categories of " Best Seller " . Replace " Best Seller " by something like " Oscars " and every " best seller " book is basically an " oscar - winning " book . May not have won the " Best film " , but even if you won the best director or best script , you 're still an " oscar


In [77]:
ai_samples = []

for text in df_ai["text"]:
    words = text.split()
    if len(words) < 50:
        continue

    ai_samples.append({
        "text": text,
        "labels": [1] * len(words)
    })

    if len(ai_samples) == 20:
        break

print("AI samples created:", len(ai_samples))


AI samples created: 20


In [78]:
sample = ai_samples[0]

print(sample["text"][:300])
print("Word count:", len(sample["text"].split()))
print("First 10 labels:", sample["labels"][:10])


Basically there are many categories of " Best Seller " . Replace " Best Seller " by something like " Oscars " and every " best seller " book is basically an " oscar - winning " book . May not have won the " Best film " , but even if you won the best director or best script , you 're still an " oscar
Word count: 267
First 10 labels: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


**Merging two sample Datasets**

In [79]:
import random

all_samples = human_samples + ai_samples
random.shuffle(all_samples)

print("Total samples:", len(all_samples))


Total samples: 40


Splitting

In [80]:
train_samples = all_samples[:32]   # 80%
val_samples = all_samples[32:]     # 20%

print(len(train_samples), len(val_samples))


32 8


Loading DE bert A

In [81]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")


/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [82]:
sample = train_samples[0]
words = sample["text"].split()
word_labels = sample["labels"]

encoding = tokenizer(
    words,                               # pass list of words
    is_split_into_words=True,
    return_offsets_mapping=True,
    padding=False,
    truncation=True,
    max_length=512                        # DeBERTa limit
)

tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
word_ids = encoding.word_ids()


In [83]:
print("WORDS:", words[:10])
print("TOKENS:", tokens[:20])
print("WORD IDs:", word_ids[:20])


WORDS: ['On', 'its', 'day', 'of', 'release', 'in', 'Japan', ',', 'Valkyria', 'Chronicles']
TOKENS: ['[CLS]', '▁On', '▁its', '▁day', '▁of', '▁release', '▁in', '▁Japan', '▁,', '▁Valk', 'yria', '▁Chronicles', '▁III', '▁topped', '▁both', '▁platform', '▁@', '-', '@', '▁exclusive']
WORD IDs: [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, 13, 14, 14, 14, 15]


In [84]:
def align_labels_with_tokens(word_labels, word_ids):
    token_labels = []
    previous_word = None
    
    for word_id in word_ids:
        if word_id is None:  
            token_labels.append(-100)  # ignore special tokens
        elif word_id != previous_word:
            token_labels.append(word_labels[word_id])  # first token of a word
            previous_word = word_id
        else:
            token_labels.append(-100)  # ignore subsequent subword tokens

    return token_labels


In [85]:
token_labels = align_labels_with_tokens(word_labels, word_ids)

print("Token labels:", token_labels[:20])
print("Tokens:", tokens[:20])
print("Word IDs:", word_ids[:20])


Token labels: [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, -100, -100, 0]
Tokens: ['[CLS]', '▁On', '▁its', '▁day', '▁of', '▁release', '▁in', '▁Japan', '▁,', '▁Valk', 'yria', '▁Chronicles', '▁III', '▁topped', '▁both', '▁platform', '▁@', '-', '@', '▁exclusive']
Word IDs: [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, 13, 14, 14, 14, 15]


In [86]:
class DebertaTokenDataset:
    def __init__(self, samples, tokenizer, max_length=512):
        self.samples = samples
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        text = self.samples[idx]["text"]
        word_labels = self.samples[idx]["labels"]
        words = text.split()
        
        encoding = self.tokenizer(
            words,
            is_split_into_words=True,
            return_offsets_mapping=True,
            padding="max_length",
            truncation=True,
            max_length=self.max_length
        )
        
        word_ids = encoding.word_ids()
        token_labels = align_labels_with_tokens(word_labels, word_ids)
        
        # convert to tensors
        item = {key: torch.tensor(val) for key, val in encoding.items() if key != "offset_mapping"}
        item["labels"] = torch.tensor(token_labels)
        
        return item


In [87]:
train_dataset = DebertaTokenDataset(train_samples, tokenizer)
val_dataset = DebertaTokenDataset(val_samples, tokenizer)


In [88]:
import torch

In [89]:
item = train_dataset[0]

print(item.keys())
print("input_ids shape:", item["input_ids"].shape)
print("labels shape:", item["labels"].shape)
print(item["labels"][:40])   # show first 40 token labels


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
input_ids shape: torch.Size([512])
labels shape: torch.Size([512])
tensor([-100,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100,    0,
           0,    0,    0,    0,    0, -100, -100,    0,    0,    0,    0, -100,
        -100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, -100, -100])


# **All the steps above was to prepare a small dataset to test and mimic a small portion of real work! This is not the real output but a demonstration of how your output will looklike according to the document you provided me**

In [90]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [91]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import torch


In [92]:
model = AutoModelForTokenClassification.from_pretrained(
    "microsoft/deberta-v3-large",
    num_labels=2,
    ignore_mismatched_sizes=True
)


Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [93]:
model


DebertaV2ForTokenClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm)

In [94]:
training_args = TrainingArguments(
    output_dir="./deberta_detector",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    fp16=True,
    logging_steps=10,
    weight_decay=0.01,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [95]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./deberta_detector",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    fp16=True,
    logging_steps=10,
    weight_decay=0.01,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [96]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./deberta_detector",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    fp16=True,
    logging_steps=10,
    weight_decay=0.01,
    report_to="none"   # 🔴 THIS IS IMPORTANT
)


In [97]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)


/tmp/ipykernel_47/1652196004.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [98]:
trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=4, training_loss=0.42701753973960876, metrics={'train_runtime': 35.6595, 'train_samples_per_second': 1.795, 'train_steps_per_second': 0.112, 'total_flos': 59437650149376.0, 'train_loss': 0.42701753973960876, 'epoch': 2.0})

In [99]:
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.14662860333919525,
 'eval_runtime': 1.4943,
 'eval_samples_per_second': 5.354,
 'eval_steps_per_second': 2.677,
 'epoch': 2.0}

In [100]:
import torch
import numpy as np

def predict_batch(texts, model, tokenizer, device="cuda"):
    model.eval()
    model.to(device)

    all_outputs = []

    with torch.no_grad():
        for text in texts:
            words = text.split()

            encoding = tokenizer(
                words,
                is_split_into_words=True,
                return_offsets_mapping=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            )

            word_ids = encoding.word_ids()
            encoding = {k: v.to(device) for k, v in encoding.items() if k != "offset_mapping"}

            outputs = model(**encoding)
            logits = outputs.logits.squeeze(0)      # [seq_len, 2]
            probs = torch.softmax(logits, dim=-1)[:, 1]  # AI probability

            # Aggregate token probs → word probs
            word_probs = {}
            for token_idx, word_id in enumerate(word_ids):
                if word_id is None:
                    continue
                word_probs.setdefault(word_id, []).append(probs[token_idx].item())

            # Use MAX over subwords (recommended)
            final_probs = [max(word_probs[i]) for i in range(len(words))]
            all_outputs.append(final_probs)

    return all_outputs


In [101]:
# Example texts
texts = [
    "This is a simple human written sentence.",
    "This text was generated by an artificial intelligence model."
]

outputs = predict_batch(texts, model, tokenizer)

print(outputs)


[[0.3232060670852661, 0.31152811646461487, 0.23936733603477478, 0.5322992205619812, 0.4280407726764679, 0.4280068278312683, 0.37719348073005676], [0.1329631358385086, 0.22803784906864166, 0.06984741985797882, 0.07881484180688858, 0.08180958032608032, 0.06731610000133514, 0.09023510664701462, 0.13098254799842834, 0.1381264179944992]]


In [102]:
words = texts[0].split()
print(words)
print(outputs[0])
print(len(words), len(outputs[0]))


['This', 'is', 'a', 'simple', 'human', 'written', 'sentence.']
[0.3232060670852661, 0.31152811646461487, 0.23936733603477478, 0.5322992205619812, 0.4280407726764679, 0.4280068278312683, 0.37719348073005676]
7 7


In [103]:
MAX_HUMAN = 10_000
human_samples = []

for item in human_ds:
    text = item["text"]
    words = text.split()

    if len(words) < 80 or len(words) > 400:
        continue

    human_samples.append({
        "text": text,
        "labels": [0] * len(words)
    })

    if len(human_samples) >= MAX_HUMAN:
        break

print("Human samples:", len(human_samples))


Human samples: 10000


In [104]:
MAX_AI = 10_000
ai_samples = []

for text in df_ai["text"]:   # or "answer"
    words = text.split()

    if len(words) < 80 or len(words) > 400:
        continue

    ai_samples.append({
        "text": text,
        "labels": [1] * len(words)
    })

    if len(ai_samples) >= MAX_AI:
        break

print("AI samples:", len(ai_samples))


AI samples: 10000


In [105]:
import torch
import numpy as np

def predict_batch(texts, model, tokenizer, device="cuda"):
    model.eval()
    model.to(device)

    all_outputs = []

    with torch.no_grad():
        for text in texts:
            words = text.split()

            encoding = tokenizer(
                words,
                is_split_into_words=True,
                return_offsets_mapping=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            )

            word_ids = encoding.word_ids()
            encoding = {k: v.to(device) for k, v in encoding.items() if k != "offset_mapping"}

            outputs = model(**encoding)
            logits = outputs.logits.squeeze(0)      # [seq_len, 2]
            probs = torch.softmax(logits, dim=-1)[:, 1]  # AI probability

            # Aggregate token probs → word probs
            word_probs = {}
            for token_idx, word_id in enumerate(word_ids):
                if word_id is None:
                    continue
                word_probs.setdefault(word_id, []).append(probs[token_idx].item())

            # Use MAX over subwords (recommended)
            final_probs = [max(word_probs[i]) for i in range(len(words))]
            all_outputs.append(final_probs)

    return all_outputs


In [106]:
# Example texts
texts = [
    "This is a simple human written sentence.",
    "This text was generated by an artificial intelligence model."
]

outputs = predict_batch(texts, model, tokenizer)

print(outputs)


[[0.3232060670852661, 0.31152811646461487, 0.23936733603477478, 0.5322992205619812, 0.4280407726764679, 0.4280068278312683, 0.37719348073005676], [0.1329631358385086, 0.22803784906864166, 0.06984741985797882, 0.07881484180688858, 0.08180958032608032, 0.06731610000133514, 0.09023510664701462, 0.13098254799842834, 0.1381264179944992]]


In [107]:
import torch, gc

torch.cuda.empty_cache()
gc.collect()

print("Memory cleared")


Memory cleared


In [108]:
import random

mixed_human_ai = []

for _ in range(800):
    h = random.choice(human_samples)
    a = random.choice(ai_samples)

    h_words = h["text"].split()
    a_words = a["text"].split()

    cut = int(0.6 * len(h_words))  # human first

    words = h_words[:cut] + a_words[:len(h_words) - cut]
    labels = [0]*cut + [1]*(len(words) - cut)

    mixed_human_ai.append({
        "text": " ".join(words),
        "labels": labels
    })


In [111]:
val_texts = [s["text"] for s in val_samples]

# sentence label: 1 if any AI word exists, else 0
val_labels = [1 if any(l == 1 for l in s["labels"]) else 0 for s in val_samples]


In [113]:
def sentence_scores(texts, model, tokenizer):
    outputs = predict_batch(texts, model, tokenizer)
    return [sum(p) / len(p) for p in outputs]


In [114]:
scores = sentence_scores(val_texts, model, tokenizer)


In [115]:
import numpy as np

best_acc = 0.0
best_threshold = 0.5

for t in np.arange(0.1, 0.9, 0.01):
    preds = [1 if s >= t else 0 for s in scores]
    acc = sum(p == y for p, y in zip(preds, val_labels)) / len(val_labels)

    if acc > best_acc:
        best_acc = acc
        best_threshold = t

print(f"Best Threshold: {best_threshold:.2f}")
print(f"Validation Accuracy: {best_acc:.3f}")


Best Threshold: 0.15
Validation Accuracy: 1.000


In [116]:
import random

mixed_human_ai = []

for _ in range(800):
    h = random.choice(human_samples)
    a = random.choice(ai_samples)

    h_words = h["text"].split()
    a_words = a["text"].split()

    cut = int(0.6 * len(h_words))  # human first

    words = h_words[:cut] + a_words[:len(h_words) - cut]
    labels = [0]*cut + [1]*(len(words) - cut)

    mixed_human_ai.append({
        "text": " ".join(words),
        "labels": labels
    })


In [117]:
mixed_ai_middle = []

for _ in range(200):
    h = random.choice(human_samples)
    a = random.choice(ai_samples)

    h_words = h["text"].split()
    a_words = a["text"].split()

    start = len(h_words) // 3
    end = start + len(h_words) // 3

    words = (
        h_words[:start]
        + a_words[: end - start]
        + h_words[end:]
    )

    labels = (
        [0]*start
        + [1]*(end - start)
        + [0]*(len(words) - end)
    )

    mixed_ai_middle.append({
        "text": " ".join(words),
        "labels": labels
    })


In [118]:
human_small = human_samples[:500]
ai_small = ai_samples[:500]

all_samples_small = (
    human_small
    + ai_small
    + mixed_human_ai
    + mixed_ai_middle
)

random.shuffle(all_samples_small)

split = int(0.8 * len(all_samples_small))
train_samples = all_samples_small[:split]
val_samples = all_samples_small[split:]

print(len(train_samples), len(val_samples))


1600 400


In [119]:
import random

# sizes for SMALL-SCALE tuning
N_HUMAN = 500
N_AI = 500
N_MIX_HAI = 800     # Human → AI
N_MIX_MID = 200     # AI in middle

# 1) pure samples
human_small = human_samples[:N_HUMAN]
ai_small = ai_samples[:N_AI]

# 2) mixed: Human → AI (40%)
mixed_human_ai = []
for _ in range(N_MIX_HAI):
    h = random.choice(human_samples)
    a = random.choice(ai_samples)

    h_words = h["text"].split()
    a_words = a["text"].split()

    cut = int(0.6 * len(h_words))
    words = h_words[:cut] + a_words[:len(h_words)-cut]
    labels = [0]*cut + [1]*(len(words)-cut)

    mixed_human_ai.append({"text": " ".join(words), "labels": labels})

# 3) mixed: AI in the middle (10%)
mixed_ai_middle = []
for _ in range(N_MIX_MID):
    h = random.choice(human_samples)
    a = random.choice(ai_samples)

    h_words = h["text"].split()
    a_words = a["text"].split()

    start = len(h_words)//3
    end = start + len(h_words)//3

    words = h_words[:start] + a_words[:end-start] + h_words[end:]
    labels = [0]*start + [1]*(end-start) + [0]*(len(words)-end)

    mixed_ai_middle.append({"text": " ".join(words), "labels": labels})

# combine
all_samples_small = (
    human_small + ai_small + mixed_human_ai + mixed_ai_middle
)

random.shuffle(all_samples_small)

print("Total small samples:", len(all_samples_small))


Total small samples: 2000


In [120]:
# 80 / 20 split
split = int(0.8 * len(all_samples_small))

train_samples = all_samples_small[:split]
val_samples = all_samples_small[split:]

print("Train samples:", len(train_samples))
print("Val samples:", len(val_samples))


Train samples: 1600
Val samples: 400


In [121]:
train_dataset = DebertaTokenDataset(train_samples, tokenizer)
val_dataset = DebertaTokenDataset(val_samples, tokenizer)

print("Datasets created")


Datasets created


In [122]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./deberta_small_tune",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,   # effective batch = 8
    num_train_epochs=5,              # more epochs for small data
    learning_rate=1e-5,              # stable learning
    fp16=True,
    logging_steps=20,
    save_total_limit=1,
    report_to="none"
)

print("TrainingArguments ready")


TrainingArguments ready


In [123]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

print("Trainer ready")


Trainer ready


/tmp/ipykernel_47/2925376803.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [141]:
import torch, gc

torch.cuda.empty_cache()
gc.collect()

print("Memory cleared")


Memory cleared


In [142]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "microsoft/deberta-v3-base",
    num_labels=2,
    ignore_mismatched_sizes=True
)

print("DeBERTa-base loaded")


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DeBERTa-base loaded


In [144]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./deberta_small_tune",
    per_device_train_batch_size=2,   # base allows this
    num_train_epochs=5,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=20,
    save_total_limit=1,
    report_to="none"
)


In [145]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

print("Trainer ready")


/tmp/ipykernel_47/2925376803.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainer ready


In [146]:
trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
20,0.683800
40,0.228500
60,0.169500
80,0.110900
100,0.019700
120,0.316200
140,0.195300
160,0.197700
180,0.111700
200,0.120700


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=2000, training_loss=0.051396172103665864, metrics={'train_runtime': 1026.5428, 'train_samples_per_second': 7.793, 'train_steps_per_second': 1.948, 'total_flos': 2090411802624000.0, 'train_loss': 0.051396172103665864, 'epoch': 5.0})

In [151]:
val_texts = [s["text"] for s in val_samples]

# Sentence label: AI if any AI word exists
val_labels = [
    1 if any(l == 1 for l in s["labels"]) else 0
    for s in val_samples
]

print("Validation samples:", len(val_texts))


Validation samples: 400


In [153]:
def sentence_scores(texts, model, tokenizer):
    outputs = predict_batch(texts, model, tokenizer)
    return [sum(p) / len(p) for p in outputs]

scores = sentence_scores(val_texts, model, tokenizer)
print("Scores computed")


Scores computed


In [154]:
import numpy as np

best_acc = 0.0
best_t = 0.5

for t in np.arange(0.1, 0.9, 0.01):
    preds = [1 if s >= t else 0 for s in scores]
    acc = sum(p == y for p, y in zip(preds, val_labels)) / len(val_labels)

    if acc > best_acc:
        best_acc = acc
        best_t = t

print(f"Best Threshold: {best_t:.2f}")
print(f"Validation Accuracy: {best_acc:.3f}")


Best Threshold: 0.10
Validation Accuracy: 1.000
